In [1]:
VER = 6

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

import os, sys, pickle, glob, gc
from collections import Counter
import cudf, itertools
print('We will use RAPIDS version',cudf.__version__)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

from pandarallel import pandarallel

pandarallel.initialize(nb_workers=4, progress_bar=True, use_memory_fs=False)

import xgboost as xgb
from sklearn.model_selection import GroupKFold

from pyarrow.parquet import ParquetFile
import pyarrow as pa 

from catboost import CatBoostRanker, Pool

We will use RAPIDS version 22.10.00a+392.g1558403753
INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


# Training

In [2]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}

In [3]:
CANDIDATE_COUNT = 100

In [4]:
RUN_FOR = "local" # "kaggle"

In [5]:
train_sessions = np.load("./splitted_raw_data/val_sessions_for_train.npy", allow_pickle=True)

In [6]:
subs = []

for type_str in tqdm(list(type_labels.keys())):
        
    batches = sorted(glob.glob(f"./candidated_features/{RUN_FOR}_{type_str}_all_data_{CANDIDATE_COUNT}candidates_p*.pqt"))

    model_paths = sorted(glob.glob(f"./models/XGB_{CANDIDATE_COUNT}candidates_fold*_{type_str}.xgb"))
#     model_paths = sorted(glob.glob(f"./models/CB_{CANDIDATE_COUNT}candidates_fold*_{type_str}.cb"))

    all_predictions = []
    
    for batch in tqdm(batches):
        whole_df = pd.read_parquet(batch)#.drop(labels=["candidate_rank"], axis=1)
        
        if RUN_FOR == "local":
            whole_df = whole_df[~whole_df.session.isin(train_sessions)].reset_index(drop=True)

        print(f"Processing {len(whole_df)} rows...")
        FEATURES = whole_df.columns[2 : -1]
        dtest = xgb.DMatrix(data=whole_df[FEATURES])
#         dtest = Pool(
#             data=whole_df[FEATURES]
#         )
        
        preds = np.zeros(len(whole_df))
        
        for model_path in model_paths:
#             target_it = int(model_path.split("_it")[-1].split(".xgb")[0])
#             model = CatBoostRanker()
#             model.load_model(model_path)
        
            model = xgb.Booster()
            model.load_model(model_path)
            model.set_param({'predictor': 'gpu_predictor'})
            
            preds += model.predict(dtest)/len(model_paths)
        
        predictions = whole_df[['session','aid']].copy()
        predictions['pred'] = preds
        all_predictions.append(predictions)
        
    all_predictions = pd.concat(all_predictions, ignore_index=True)
    
    all_predictions = all_predictions.sort_values(['session','pred'],
                                                  ascending=[True,False]).reset_index(drop=True)
    all_predictions['n'] = all_predictions.groupby('session').aid.cumcount().astype('int8')
    all_predictions = all_predictions.loc[all_predictions.n<20]

    sub = all_predictions.groupby('session').aid.apply(list)
    sub = sub.to_frame().reset_index()
    sub.item = sub.aid.apply(lambda x: " ".join(map(str,x)))
    sub.columns = ['session_type','labels']
    sub.session_type = sub.session_type.astype('str') + '_' + type_str

    subs.append(sub)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

Processing 6069400 rows...
Processing 6062200 rows...
Processing 6050500 rows...
Processing 6065900 rows...
Processing 6077600 rows...
Processing 6071600 rows...
Processing 6081400 rows...
Processing 6046100 rows...
Processing 6048500 rows...
Processing 6067400 rows...
Processing 77400 rows...
Processing 6079900 rows...
Processing 6081100 rows...
Processing 6088300 rows...
Processing 6068800 rows...
Processing 6062400 rows...
Processing 6071500 rows...
Processing 6053200 rows...
Processing 6055300 rows...


/tmp/ipykernel_776484/2417223045.py:51: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sub.item = sub.aid.apply(lambda x: " ".join(map(str,x)))


  0%|          | 0/19 [00:00<?, ?it/s]

Processing 6069400 rows...
Processing 6062200 rows...
Processing 6050500 rows...
Processing 6065900 rows...
Processing 6077600 rows...
Processing 6071600 rows...
Processing 6081400 rows...
Processing 6046100 rows...
Processing 6048500 rows...
Processing 6067400 rows...
Processing 77400 rows...
Processing 6079900 rows...
Processing 6081100 rows...
Processing 6088300 rows...
Processing 6068800 rows...
Processing 6062400 rows...
Processing 6071500 rows...
Processing 6053200 rows...
Processing 6055300 rows...


/tmp/ipykernel_776484/2417223045.py:51: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sub.item = sub.aid.apply(lambda x: " ".join(map(str,x)))


  0%|          | 0/19 [00:00<?, ?it/s]

Processing 6069400 rows...
Processing 6062200 rows...
Processing 6050500 rows...
Processing 6065900 rows...
Processing 6077600 rows...
Processing 6071600 rows...
Processing 6081400 rows...
Processing 6046100 rows...
Processing 6048500 rows...
Processing 6067400 rows...
Processing 77400 rows...
Processing 6079900 rows...
Processing 6081100 rows...
Processing 6088300 rows...
Processing 6068800 rows...
Processing 6062400 rows...
Processing 6071500 rows...
Processing 6053200 rows...
Processing 6055300 rows...


/tmp/ipykernel_776484/2417223045.py:51: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  sub.item = sub.aid.apply(lambda x: " ".join(map(str,x)))


## Local Score

In [7]:
final_sub = pd.concat(subs, ignore_index=True)
final_sub.sort_values(by="session_type", ascending=True).reset_index(drop=True)

if RUN_FOR == "local":
    # COMPUTE METRIC
    score = 0
    weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
    for t in ['clicks','carts','orders']:
        sub = final_sub.loc[final_sub.session_type.str.contains(t)].copy()
        sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
    #     sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
        test_labels = pd.read_parquet('./splitted_raw_data/val_labels.parquet')
        test_labels = test_labels[~test_labels.session.isin(train_sessions)].reset_index(drop=True)
        test_labels = test_labels.loc[test_labels['type']==t]
        test_labels = test_labels.merge(sub, how='left', on=['session'])
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
        score += weights[t]*recall
        print(f'{t} recall =',recall)

    print('=============')
    print('Overall Recall =',score)
    print('=============')

elif RUN_FOR == "kaggle":
    final_sub["labels"] = final_sub.labels.apply(lambda x: " ".join([str(elm) for elm in x]))
#     final_sub.to_csv("submission.csv", index=False)
    final_sub.to_csv("submission.csv.gz", index=False, compression='gzip')

clicks recall = 0.5327735558171982
carts recall = 0.42333887304524925
orders recall = 0.6599005864871262
Overall Recall = 0.5762193693875703


In [8]:
final_sub

,session_type,labels
0,11098528_clicks,"[11830, 588923, 1732105, 571762, 876129, 88450..."
1,11098529_clicks,"[1105029, 459126, 217742, 1339838, 1694360, 15..."
2,11098530_clicks,"[409236, 264500, 963957, 1603001, 583026, 2541..."
3,11098533_clicks,"[1074173, 1165015, 385390, 1309900, 978918, 76..."
4,11098534_clicks,"[223062, 908024, 1342293, 1607945, 1039538, 12..."
...,...,...
3278350,12899770_orders,"[277173, 480578, 746786, 1602968, 1457452, 143..."
3278351,12899772_orders,"[1062165, 1234522, 878643, 346692, 1339305, 51..."
3278352,12899773_orders,"[1311526, 1484665, 37930, 337571, 104552, 1255..."
3278353,12899774_orders,"[33035, 1584669, 255895, 819288, 1089446, 8517..."
